## L'audio

### Text to speech

In [ ]:
from openai import OpenAI

response = OpenAI().audio.speech.create(
  model="tts-1-hd",
  voice="onyx", # alloy, onyx, fable, echo
  input="Coucou, comment allez-vous ?"
)
response.with_streaming_response.method('mon_audio.mp3')

### Speech to text

In [ ]:
from openai import OpenAI

OpenAI().audio.transcriptions.create(
  model="whisper-1", 
  file=open("mon_audio.mp3", "rb"),
  language="fr"
).text

In [ ]:
from openai import OpenAI
from pydub import AudioSegment

client = OpenAI()

song = AudioSegment.from_mp3("mon_fichier.mp3")
transcription = ''
for pas in range(0, 120, 20):
    debut, fin = pas * 60 * 1000, (pas+20) * 60 * 1000
    extrait = song[debut:fin]
    if len(extrait) > 100:
        extrait.export(f"extrait_{pas}.mp3", format="mp3")
        audio_file = open(f"extrait_{pas}.mp3", "rb")
        transcription += client.audio.transcriptions.create(
          model="whisper-1", 
          file=audio_file
        ).text

## Multimodal

In [33]:
import base64

def encode_image_to_data_url(image_path: str) -> str:
    """
    Lit l'image et renvoie une data URL prête à être insérée dans le payload.
    """
    # Lecture du fichier image
    with open(image_path, "rb") as img_file:
        image_bytes = img_file.read()  # Lecture binaire du contenu
    # Encodage base64
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    # Détection du MIME type selon l'extension
    ext = os.path.splitext(image_path)[1].lower().lstrip(".")
    mime = f"image/{ext if ext != 'jpg' else 'jpeg'}"
    return f"data:{mime};base64,{b64}"

In [34]:
from openai import OpenAI

client = OpenAI()

data_url = encode_image_to_data_url("mon_image.png")
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Que vois-tu sur cette image ?"},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    }
]

NameError: name 'os' is not defined

In [ ]:
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    max_tokens=500,
    temperature=0.0
)
response.choices[0].message.content